In [1]:
# NumPy
import numpy as np

# Dataframe operations
import pandas as pd

# XGBoost
import xgboost as xgb

# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Scalers
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

#dataset
from sklearn.datasets import load_boston

# Models
from sklearn.linear_model import LogisticRegression #logistic regression
from sklearn.linear_model import Perceptron
from sklearn import svm #support vector Machine
from sklearn.ensemble import RandomForestClassifier #Random Forest
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn.naive_bayes import GaussianNB #Naive bayes
from sklearn.tree import DecisionTreeClassifier #Decision Tree
from sklearn.model_selection import train_test_split #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix #for confusion matrix
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

# Cross-validation
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

# GridSearchCV
from sklearn.model_selection import GridSearchCV

#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix
import warnings
import math
warnings.filterwarnings('ignore')

In [23]:
# CSVを読み込む
test= pd.read_csv("test.csv")
train= pd.read_csv("train.csv")

# データの統合
data = pd.concat([train, test], ignore_index = True)

# 提出用に
row_id = test["row_id"]

data['date'] = data['time'].str[:10]
data["time"] = data['time'].str[-8:-3]
data['date'] = pd.to_datetime(data['date'])
data.dtypes


row_id                 int64
time                  object
x                      int64
y                      int64
direction             object
congestion           float64
date          datetime64[ns]
dtype: object

In [44]:
data1 = data[['time','x','y','direction','congestion','date']]
data1['date'] = data1['date'].dt.weekday
data1_dummies=pd.get_dummies(data1)
data1_dummies

,x,y,congestion,date,time_00:00,time_00:20,time_00:40,time_01:00,time_01:20,time_01:40,...,time_23:20,time_23:40,direction_EB,direction_NB,direction_NE,direction_NW,direction_SB,direction_SE,direction_SW,direction_WB
0,0,0,70.0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,49.0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,24.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,1,18.0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,1,60.0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851170,2,3,NaN,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
851171,2,3,NaN,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
851172,2,3,NaN,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
851173,2,3,NaN,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [45]:
model_train = data1_dummies[:848835]
model_test = data1_dummies[848835:]

X = model_train.drop('congestion', axis=1)
Y = pd.DataFrame(model_train['congestion'])
x_test = model_test.drop('congestion', axis=1)
model_train.head()

,x,y,congestion,date,time_00:00,time_00:20,time_00:40,time_01:00,time_01:20,time_01:40,...,time_23:20,time_23:40,direction_EB,direction_NB,direction_NE,direction_NW,direction_SB,direction_SE,direction_SW,direction_WB
0,0,0,70.0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,49.0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,24.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,1,18.0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,1,60.0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [46]:
params = {'objective':'binary:logistic',
          'max_depth':5,
          'eta': 0.1, 
          'min_child_weight':1.0,
          'gamma':0.0,
          'colsample_bytree':0.8}

num_round = 1000

logloss = []
accuracy = []

kf = KFold(n_splits=4, shuffle=True, random_state=0)
for train_index, valid_index in kf.split(X):
    x_train, x_valid = X.iloc[train_index], X.iloc[valid_index] 
    y_train, y_valid = Y.iloc[train_index], Y.iloc[valid_index]
    
    dtrain = xgb.DMatrix(x_train, label=y_train)
    dtest = xgb.DMatrix(x_valid, label=y_valid)
    
    xgb_params = {
        # 回帰問題
        'objective': 'reg:linear',
        # 学習用の指標 (RMSE)
        'eval_metric': 'rmse',
        'tree_method':'gpu_hist'
    }
    evals = [(dtrain, 'train'), (dtest, 'eval')]
    evals_result = {}
    bst = xgb.train(xgb_params,
                    dtrain,
                    num_boost_round=50000,
                    early_stopping_rounds=15,
                    evals=evals,
                    evals_result=evals_result,
                    verbose_eval=200
                    )

    y_pred = bst.predict(dtest)
    mse = mean_squared_error(y_valid, y_pred)
    print('RMSE:', math.sqrt(mse))
    
    

[13:09:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:36.51748	eval-rmse:36.56674
[200]	train-rmse:9.23531	eval-rmse:9.32654
[400]	train-rmse:9.01326	eval-rmse:9.14801
[600]	train-rmse:8.91253	eval-rmse:9.07761
[800]	train-rmse:8.85142	eval-rmse:9.04162
[1000]	train-rmse:8.81187	eval-rmse:9.02300
[1200]	train-rmse:8.78251	eval-rmse:9.01127
[1400]	train-rmse:8.76104	eval-rmse:9.00520
[1600]	train-rmse:8.74348	eval-rmse:9.00186
[1713]	train-rmse:8.73512	eval-rmse:9.00009
RMSE: 9.000062783185163
[13:10:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:36.53255	eval-rmse:36.51585
[200]	train-rmse:9.25764	eval-rmse:9.34370
[400]	train-rmse:9.00575	eval-rmse:9.13825
[600]	train-rmse:8.90732	eval-rmse:9.07419
[800]	train-rms

In [47]:
#predictで予測
dtest = xgb.DMatrix(x_test)
y_pred = bst.predict(dtest)
submission = pd.DataFrame({'row_id':test['row_id'], 'congestion':y_pred})
submission.to_csv('submission3.csv', index=False)